In [1]:
import pyspark
from delta import *
from pyspark.sql import functions as sf
from delta.tables import *

builder = (
    pyspark.sql.SparkSession.builder.appName("MyApp")
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config(
        "spark.sql.catalog.spark_catalog",
        "org.apache.spark.sql.delta.catalog.DeltaCatalog",
    )
)

table = "./tmp/delta-table"
spark = configure_spark_with_delta_pip(builder).getOrCreate()


25/05/14 23:38:01 WARN Utils: Your hostname, wagner-VirtualBox resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
25/05/14 23:38:01 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/wagner/Documents/delta_test/.venv/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/wagner/.ivy2/cache
The jars for the packages stored in: /home/wagner/.ivy2/jars
io.delta#delta-spark_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-f5615484-5bb7-4f90-85ff-23c8d4564195;1.0
	confs: [default]
	found io.delta#delta-spark_2.12;3.3.1 in central
	found io.delta#delta-storage;3.3.1 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 411ms :: artifacts dl 29ms
	:: modules in use:
	io.delta#delta-spark_2.12;3.3.1 from central in [default]
	io.delta#delta-storage;3.3.1 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0

In [ ]:
df2 = spark.read.format("parquet").load(
    "/home/wagner/Documents/delta_test/tmp/delta-table/_delta_log/00000000000000000010.checkpoint.parquet"
)

df2.filter(sf.col("remove").isNotNull()).show(n=10, truncate=False, vertical=True)
df2.printSchema()




-RECORD 0--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 txn            | NULL                                                                                                                                                                               
 add            | NULL                                                                                                                                                                               
 remove         | {date_partition=2025-05-15/part-00001-772333d4-efea-47ea-ad61-6a91cb262600.c000.snappy.parquet, 1747279858351, false, true, {date_partition -> 2025-05-15}, 478, NULL, NULL, NULL} 
 metaData       | NULL                                                                                                                                                                               
 protocol 

In [21]:
partitions_modified = set()
for row in df2.collect():
    if row["add"]:
        timestamp_column = "modificationTime"
        partition = set(row["add"]["partitionValues"].values())
        
    if row["remove"]:
        timestamp_column = "deletionTimestamp"
        partition = set(row["remove"]["partitionValues"].values())

    #print(partition)
    partitions_modified = partitions_modified.union(partition)
    

In [22]:
print(partitions_modified)


{'2025-05-14', '2025-05-15'}
